#### 단어 임베딩 : 각 단어를 벡터로 표현. 
####             : - 동음이의어 구분X -> 동일한 벡터값으로 표현.
####             : + 문장 임베딩에 비해 학습 방법 간단.

In [ ]:
# 1. 원-핫 인코딩(one-hot encoding)
# -> 원-핫 벡터 = 희소(sparse) 벡터 = 희소 행렬 : 전체 요소 중 단 하나의 값만 1

### 과정
# pip install konlpy
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = '오늘 날씨는 구름이 많아요.'

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [ ]:
# 단어 사전 구축 
dics = {}
for word in nouns:
  if word not in dics.keys():
    dics[word] = len(dics) # 단어별 인덱스 부여
print(dics)

{'오늘': 0, '날씨': 1, '구름': 2}


In [ ]:
# 원-핫 인코딩
nb_classes = len(dics) # 원-핫 벡터 차원의 크기 = 단어 수
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [21]:
# 2. Word2Vec : 분산 표현(distributed representation) = 밀집(dense) 표현

from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

def read_review_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # 헤더 제외
  return data

# 학습 시간 측정 시작
start = time.time() 

review_data = read_review_data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('소요 시간 :', time.time() - start) # 초

200000
소요 시간 : 0.6388688087463379


In [22]:
komoran = Komoran()

# 명사만 추출
docs = [komoran.nouns(sentence[1]) for sentence in review_data]

# 모델 학습
model = Word2Vec(sentences=docs, # 학습에 필요한 문장 데이터, 모델의 입력값
                 size=200,       # 벡터의 차원, 크기
                 window=4,       # 주변 단어 크기 (-+4)
                 hs=1,           # softmax 사용
                 min_count=2,    # 빈도수 2이하의 단어들은 학습X
                 sg=1)           # skip_gram 모델 <-> 0 : CBOW 모델

# 모델 저장
model.save('nvmc_model')

print('학습된 전체 말뭉치 수 :', model.corpus_count)
print('전체 단어 수 :', model.corpus_total_words)


학습된 전체 말뭉치 수 : 200000
전체 단어 수 : 1076896


In [23]:
# model = Word2Vec.load('nvmc.model')
print(model.corpus_total_words)

1076896


In [24]:
# '사랑'의 단어 임베딩 벡터
print('사랑 :', model.wv['사랑'])

사랑 : [-3.25132996e-01  9.49239358e-02 -7.27859065e-02  3.75660732e-02
  1.52383685e-01 -1.54868037e-01  9.90684330e-02  7.32056946e-02
  5.43037839e-02  1.58139408e-01  1.59347922e-01 -5.50184011e-01
  1.82143494e-01 -2.21779034e-01  9.28486437e-02  2.30329752e-01
  8.90698507e-02 -2.83395588e-01  2.82393664e-01 -6.60710409e-02
 -3.43848675e-01  2.35406503e-01  2.15733856e-01 -2.59648144e-01
  6.88189194e-02 -2.94554383e-01 -6.33020885e-03  1.12724416e-01
 -6.83600008e-02 -2.57834904e-02  2.23865554e-01 -2.60134898e-02
  1.07018173e-01 -2.69229501e-01  1.64983153e-01  2.97880501e-01
  2.04045057e-01 -2.30742767e-01 -1.96260989e-01  5.65744378e-02
  2.28106335e-01 -1.48322344e-01  9.81271565e-02 -1.00983381e-01
  3.92448992e-01 -1.25985697e-01 -1.09778628e-01 -3.06098133e-01
  9.04968306e-02 -3.99273843e-01  5.41122109e-02  1.99968666e-01
 -3.78118758e-03 -2.42162757e-02  1.15088336e-01  1.05973417e-02
 -1.40424982e-01  2.24841192e-01 -7.63004273e-02  3.22514564e-01
 -2.70276755e-01 -2.

In [25]:
# 단어 유사도 계산
print('일요일 = 월요일 :', model.wv.similarity(w1='일요일', w2='월요일'))
print('안성기 = 배우 :', model.wv.similarity(w1='안성기', w2='배우'))
print('대기업 = 삼성 :', model.wv.similarity(w1='대기업', w2='삼성'))
print('일요일 = 삼성 :', model.wv.similarity(w1='일요일', w2='삼성'))
print('히어로 = 삼성 :', model.wv.similarity(w1='히어로', w2='삼성'))

일요일 = 월요일 : 0.6485686
안성기 = 배우 : 0.53805023
대기업 = 삼성 : 0.59950507
일요일 = 삼성 : 0.28219217
히어로 = 삼성 : 0.116313495


In [26]:
# 가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn=5))
print(model.wv.most_similar('시리즈', topn=5))

[('장미희', 0.7327877283096313), ('강수연', 0.7213166952133179), ('씨야', 0.7197945713996887), ('김갑수', 0.705034077167511), ('정려원', 0.6993281841278076)]
[('캐리비안의 해적', 0.6782627701759338), ('잭 라이언', 0.6648984551429749), ('더 울버린', 0.6647076606750488), ('러시아워', 0.6503764390945435), ('반지의 제왕', 0.6404792666435242)]
